In [16]:
import ROOT

In [17]:
def linspace(start, end, steps):
    values = []
    for i in range(steps):
        values.append(start + i*(end-start)/(steps-1))
    return values

In [20]:
real_signal_mu = 100
real_signal_sigma = 20

model_signal_mu_spread_interval_length = 20 * 2
model_signal_mu_step_count = 5

model_signal_mus = linspace(real_signal_mu-model_signal_mu_spread_interval_length/2, real_signal_mu + model_signal_mu_spread_interval_length/2, model_signal_mu_step_count)
model_signal_sigma = real_signal_sigma

treshold_lower = 0
treshold_upper = 60
treshold_step_count = 5
treshold_values =  linspace(treshold_lower, treshold_upper, treshold_step_count)

noise_mu = 0
noise_sigma = 5

print(model_signal_mus)
print(treshold_values)

model_noise = ROOT.TF1("ElecNoiseBg", "TMath::Gaus(x, %f, %f)" % (noise_mu, noise_sigma), -100, 500)
model_noise.SetNpx(10000)


"""
# more paramaters
simulation_time = 1 #seconds
bunch_crossings_per_second = 40 * 1e6
collisions_per_bunch_crossing = en fördelning
particles_per_collision = en fördelning
particles_reaching_the_sensor = en fördelning vi inte känner till och inte är vår uppgift att känna till
 
"""

prng = ROOT.TRandom3(123)

for model_signal_mu in model_signal_mus:
    model_signal = ROOT.TF1("Signal", "TMath::Landau(x, %f, %f)" % (model_signal_mu, model_signal_sigma), -100, 500)
    model_signal.SetNpx(10000)
    for treshold_value in treshold_values:
        model_expected_signal_above = 1 - model_signal.Integral(-100, treshold_value)/model_signal.Integral(-100, 500)
        model_expected_noise_above = 1 - model_noise.Integral(-100, treshold_value)/model_noise.Integral(-100, 500)
        print(model_expected_signal_above)
        print(model_expected_noise_above)
        
        event_count = 10000
        event_ratio = 0.5
        signal_count = int(round(event_count * event_ratio))
        noise_count = event_count - signal_count

        noise_above_count = 0
        signal_above_count = 0

        for i in range(signal_count):
            signal = prng.Landau(real_signal_mu, real_signal_sigma)
            if signal > treshold_value:
                signal_above_count += 1

        for i in range(noise_count):
            noise = prng.Gaus(noise_mu, noise_sigma)
            if noise > treshold_value:
                noise_above_count += 1
                
        total_above_count = noise_above_count + signal_above_count
        
        """
        Antag att vi vet:
            total_above_count
            event_ratio
            model_expected_signal_above
            model_expected_noise_above
        
        Hur ska vi räkna?
        Vi vill beräkna ett värde som är så nära signal_count som möjligt.
        """
        
        
        
        
        

[80.0, 90.0, 100.0, 110.0, 120.0]
[0.0, 15.0, 30.0, 45.0, 60.0]
0.9999999998258093
0.5000000000000002
0.999990068010805
0.0013498980316301035
0.9979268024497532
9.865879224690843e-10
0.9700093451594182
0.0
0.8799406377446929
3.3306690738754696e-16
0.9999999999999997
0.5000000000000002
0.9999999832893397
0.0013498980316301035
0.9999090710455358
9.865879224690843e-10
0.9937793426307107
0.0
0.9472718522304674
3.3306690738754696e-16
1.0
0.5000000000000002
0.9999999999994842
0.0013498980316301035
0.9999994007941588
9.865879224690843e-10
0.9994778858416928
0.0
0.9850610323963953
3.3306690738754696e-16
1.0
0.5000000000000002
1.0
0.0013498980316301035
0.9999999998249957
9.865879224690843e-10
0.9999900216154641
0.0
0.9979171179137326
3.3306690738754696e-16
1.0
0.5000000000000002
1.0
0.0013498980316301035
0.9999999999999997
9.865879224690843e-10
0.9999999832070137
0.0
0.9999086230791328
3.3306690738754696e-16
